In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
train=pd.read_csv('/content/drive/My Drive/Train(8).csv', usecols=["safe_text", "label"])
test_df=pd.read_csv('/content/drive/My Drive/Test(6).csv', usecols=["safe_text"])
feat_cols = "safe_text"
label_cols = "label"
train.head()

,safe_text,label
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0
1,I'm 100% thinking of devoting my career to pro...,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0
3,I mean if they immunize my kid with something ...,-1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0


In [0]:
test_df.head()

,safe_text
0,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,Students starting school without whooping coug...
2,"I'm kinda over every ep of <user> being ""rippe..."
3,How many innocent children die for lack of vac...
4,"CDC eyeing bird flu vaccine for humans, though..."


In [0]:
test_df[feat_cols] = test_df[feat_cols].astype(str)
train[feat_cols] = train[feat_cols].astype(str)

In [0]:
train['label'].isna().sum().sum()

1

In [0]:
train=train.dropna()

In [0]:
train.head()

,safe_text,label
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0
1,I'm 100% thinking of devoting my career to pro...,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0
3,I mean if they immunize my kid with something ...,-1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0


In [0]:
test_df=test_df.fillna('nan')
#train=train.fillna('nan')


In [0]:
train.shape,test_df.shape

((10000, 2), (5177, 1))

In [0]:
import re
def clean_text(text):
    #Remove emojis and special chars
    clean=text
    #reg = re.compile('\\.+?(?=\B|$)')
    #clean = text.apply(lambda r: re.sub(reg, string=r, repl=''))
    #reg = re.compile('\x89Û_')
    #clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    reg = re.compile('\&amp')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
    reg = re.compile('\\n')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))

    #Remove hashtag symbol (#)
    #clean = clean.apply(lambda r: r.replace('#', ''))

    #Remove user names
    reg = re.compile('@[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='@'))

    #Remove URLs
    reg = re.compile('https?\S+(?=\s|$)')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='www'))

    #Lowercase
    #clean = clean.apply(lambda r: r.lower())
    return clean
train['safe_text'] = clean_text(train['safe_text'])
test_df['safe_text'] = clean_text(test_df['safe_text'])

In [0]:
train.head()

,safe_text,label
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0
1,I'm 100% thinking of devoting my career to pro...,1.0
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0
3,I mean if they immunize my kid with something ...,-1.0
4,Thanks to <user> Catch me performing at La Nui...,0.0


In [0]:
! pip install pytorch-transformers

     |████████████████████████████████| 184kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 15.6MB/s 
     |████████████████████████████████| 890kB 25.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=669ab56426e334ea2d8635e41184f5ec45746cb6d8bb8c125e09c093c08df211
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from fastai.text import *
from fastai.metrics import *
from pytorch_transformers import RobertaTokenizer

In [0]:
# Creating a config object to store task specific information
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=False,
    seed = 2019,
    roberta_model_name='roberta-large', # can also be exchnaged with roberta-large 
    max_lr=1e-5,
    epochs=2,
    use_fp16=False,
    bs=4, 
    max_seq_len=96, 
    num_labels = 3,
    hidden_dropout_prob=.05,
    hidden_size=1024, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
)

In [0]:
train.dtypes

safe_text     object
label        float64
dtype: object

In [0]:
class FastAiRobertaTokenizer(BaseTokenizer):
    """Wrapper around RobertaTokenizer to be compatible with fastai"""
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        """Adds Roberta bos and eos tokens and limits the maximum sequence length""" 
        return [config.start_tok] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + [config.end_tok]

In [0]:
# create fastai tokenizer for roberta
roberta_tok = RobertaTokenizer.from_pretrained("roberta-large")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

100%|██████████| 456318/456318 [00:00<00:00, 1813552.65B/s]


In [0]:
# create fastai vocabulary for roberta
path = Path()
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

In [0]:
# Setting up pre-processors
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=fastai_roberta_vocab, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(vocab=vocab)]


In [0]:
# Creating a Roberta specific DataBunch class
class RobertaDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training Roberta"
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)


In [0]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [0]:
test_df.shape

(5177, 1)

In [0]:
sub=pd.read_csv('/content/drive/My Drive/SampleSubmission(18).csv')


In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=999) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub1.csv',index=False)


100%|██████████| 1425941629/1425941629 [00:32<00:00, 44488165.13B/s]


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.344478,0.310786,0.491188,05:28
1,0.224179,0.230973,0.397154,05:30
2,0.199767,0.213947,0.373155,05:29


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=30000) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds2,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds2

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub2.csv',index=False)


epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.297060,0.360069,0.536846,05:27
1,0.225564,0.216555,0.390990,05:29
2,0.149344,0.222103,0.376802,05:30


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=42) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds3,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds3

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub3.csv',index=False)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.286233,0.320767,0.499708,05:28
1,0.200946,0.324632,0.475297,05:30
2,0.205122,0.227666,0.377857,05:29


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
sub1=pd.read_csv('/content/sentimentsub1.csv')
sub2=pd.read_csv('/content/sentimentsub2.csv')
sub3=pd.read_csv('/content/sentimentsub3.csv')
sub1['label']=0.35*sub1['label']+0.3*sub2['label']+0.35*sub3['label']
sub1.to_csv('sentimentsub_ens.csv',index=False)#0.5040

In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=2020) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds4,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds4

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub4.csv',index=False)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.310462,0.310225,0.495539,05:29
1,0.220305,0.241209,0.436551,05:29
2,0.167402,0.239809,0.396257,05:32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=80000) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds5,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds5

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub5.csv',index=False)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.285385,0.278302,0.472966,05:30
1,0.253556,0.224209,0.391467,05:28
2,0.153945,0.230735,0.389740,05:28


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)
data = RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor) \
    .split_by_rand_pct(seed=666) \
    .label_from_df(cols=label_cols,label_cls=FloatList) \
    .add_test(RobertaTextList.from_df(test_df, ".", cols=feat_cols, processor=processor)) \
    .databunch(bs=4, pad_first=False, pad_idx=0)


import torch
import torch.nn as nn
from pytorch_transformers import RobertaModel

# defining our model architecture 
class CustomRobertaModel(nn.Module):
    def __init__(self,num_labels=1):
        super(CustomRobertaModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaModel.from_pretrained(config.roberta_model_name)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels) # defining final output layer
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _ , pooled_output = self.roberta(input_ids, token_type_ids, attention_mask) # 
        logits = self.classifier(pooled_output)        
        return logits
roberta_model = CustomRobertaModel()

learn = Learner(data, roberta_model, metrics=[rmse])

learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(3, max_lr=config.max_lr)


def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

test_preds6,preds = get_preds_as_nparray(DatasetType.Test)
sub['label']=test_preds6

for e,p in enumerate(sub['label']):
  if p<-1:
    sub['label'][e]=-1

for e,p in enumerate(sub['label']):
  if p>1:
    sub['label'][e]=1

sub.to_csv('sentimentsub6.csv',index=False)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,0.272258,0.400669,0.544537,05:27
1,0.236850,0.231394,0.402734,05:28
2,0.109465,0.226802,0.384359,05:27


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
sub4=pd.read_csv('/content/sentimentsub4.csv')
sub5=pd.read_csv('/content/sentimentsub5.csv')
sub6=pd.read_csv('/content/sentimentsub6.csv')
sub4['label']=0.3*sub4['label']+0.3*sub5['label']+0.4*sub6['label']
sub4.to_csv('sentimentsub_ens1.csv',index=False)#0.50120

In [0]:
sub11=pd.read_csv('/content/sentimentsub_ens.csv')
sub22=pd.read_csv('/content/sentimentsub_ens1.csv')
sub11['label']=0.5*sub11['label']+0.5*sub22['label']
sub11.to_csv('final_sub.csv',index=False)#0.49818

In [0]:
#Lstm approach

In [6]:
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
# Setup
!pip install -q wordcloud
import wordcloud
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import pandas as pd
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import RMSprop,SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
df=pd.read_csv('/content/Train (10).csv')

df = df.loc[df['agreement'] > 0.34]
df=df.dropna()
Y = df.label

# Print initial shape
print(Y.shape)
X=df.safe_text


max_len = 200
# Build the dictionary of indexes
tok = Tokenizer()
tok.fit_on_texts(X)
# Change texts into sequence of indexes
sequences = tok.texts_to_sequences(X)
# Pad the sequences
sequences_matrix_train = sequence.pad_sequences(sequences,maxlen=max_len)



test=pd.read_csv('/content/Test (5).csv')
test=test.safe_text
test=test.fillna(method='ffill')
#same preprocessing for test 

tok.fit_on_texts(test)
sequences = tok.texts_to_sequences(test)
sequences_matrix_test = sequence.pad_sequences(sequences,maxlen=max_len)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
(9760,)


In [31]:
#choosing vocabulary for embedding layer 
max_words=2000000
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Dropout(0.5)(layer)
    layer = (Conv1D(filters=64,
                 kernel_size=4,
                 padding='valid',
                 activation='sigmoid',
                 strides=1))(layer)
    layer = (MaxPooling1D(pool_size=2))(layer)
    layer = Dropout(0.5)(layer)
    layer = LSTM(64)(layer)
    
    
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('tanh')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model 
model = RNN()
model.summary()
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 200, 50)           100000000 
_________________________________________________________________
dropout_12 (Dropout)         (None, 200, 50)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 197, 64)           12864     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 98, 64)            0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 98, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                3302

In [32]:
model.fit(sequences_matrix_train,Y,batch_size=128,epochs=8,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7808 samples, validate on 1952 samples
Epoch 1/8
7808/7808 [==============================] - 13s 2ms/step - loss: 0.4033 - accuracy: 0.4982 - val_loss: 0.3759 - val_accuracy: 0.5020
Epoch 2/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.3939 - accuracy: 0.5022 - val_loss: 0.3754 - val_accuracy: 0.5020
Epoch 3/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.3871 - accuracy: 0.5063 - val_loss: 0.3565 - val_accuracy: 0.5666
Epoch 4/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.3399 - accuracy: 0.6119 - val_loss: 0.3136 - val_accuracy: 0.6675
Epoch 5/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.2881 - accuracy: 0.7011 - val_loss: 0.3107 - val_accuracy: 0.6685
Epoch 6/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.2513 - accuracy: 0.7432 - val_loss: 0.3090 - val_accuracy: 0.6767
Epoch 7/8
7808/7808 [==============================] - 12s 2ms/step - loss: 0.2252 - acc

In [0]:
p=model.predict(sequences_matrix_test)


In [34]:
p

array([[-0.10419193],
       [ 0.2455794 ],
       [ 0.03512662],
       ...,
       [ 0.7977812 ],
       [ 0.00842187],
       [ 0.42403623]], dtype=float32)

In [35]:
#choosing vocabulary for embedding layer 
max_words=2000000
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    

    layer = LSTM(64)(layer)
    
    
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('tanh')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model 
model = RNN()
model.summary()
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 50)           100000000 
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_9 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [36]:
model.fit(sequences_matrix_train,Y,batch_size=128,epochs=2,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7808 samples, validate on 1952 samples
Epoch 1/2
7808/7808 [==============================] - 22s 3ms/step - loss: 0.3722 - accuracy: 0.5499 - val_loss: 0.3008 - val_accuracy: 0.6716
Epoch 2/2
7808/7808 [==============================] - 21s 3ms/step - loss: 0.2479 - accuracy: 0.7367 - val_loss: 0.2958 - val_accuracy: 0.6875


In [0]:
q=model.predict(sequences_matrix_test)
a=q*2/3+p*1/3

In [0]:
sub=pd.read_csv('/content/SampleSubmission (11).csv')
sub['label']=a
final=pd.read_csv('/content/final_sub.csv')
final['label']=final['label']*0.85+sub['label']*0.15
final.to_csv('se.csv',index=False)

In [0]:
#0.487578702571676